In [43]:
import pandas as pd
import sqlite3

## 1. Create Database

In [44]:
conn = sqlite3.connect("readmissions.db")

## 2. Open Cursor

In [45]:
cursor = conn.cursor()

## 3. Create Raw Table

In [46]:
with open("../sql/02_1_create_diabetes_readmissions_raw.sql", "r") as f:
    sql_script = f.read()

print("Create raw table SQL:", sql_script, sep="\n")
cursor.executescript(sql_script)

cursor.executescript(sql_script)

Create raw table SQL:
-- RAW TABLE - Stores original diabetes readmissions data without transforms.
  
DROP TABLE IF EXISTS diabetes_readmissions;

CREATE TABLE diabetes_readmissions
(
    encounter_id NUMBER,
    patient_nbr NUMBER,
    race VARCHAR2 (100),
    gender VARCHAR2 (100),
    age VARCHAR2 (100),
    admission_type_id NUMBER,
    discharge_disposition_id NUMBER,
    time_in_hospital NUMBER,
    num_lab_procedures NUMBER,
    num_medications NUMBER,
    number_diagnoses NUMBER,
    diag_1 VARCHAR2(100),
    diag_2 VARCHAR2(100),
    diag_3 VARCHAR2(100),
    readmitted VARCHAR2(100),
    admission_source_id NUMBER,     
    medical_specialty VARCHAR2(100),
    diabetesMed VARCHAR2(100),     
    insulin VARCHAR2(100),       
    change VARCHAR2(100),
    num_procedures NUMBER,
    number_outpatient NUMBER,
    number_emergency NUMBER,
    number_inpatient NUMBER,
    A1Cresult VARCHAR2(100),
    max_glu_serum VARCHAR2(100));
    
-- Rename column to avoid reserved-word ambig

## 4. Insert Data into Raw Table

In [47]:
with open("../sql/02_2_insert_diabetes_readmission_raw.sql", "r") as f:
    sql_script = f.read()

print("Insert raw data SQL:", sql_script, sep="\n")

cursor.executescript(sql_script)

Insert raw data SQL:
DELETE FROM diabetes_readmissions;
INSERT INTO diabetes_readmissions (encounter_id, patient_nbr, race, gender, age, admission_type_id, discharge_disposition_id, time_in_hospital, num_lab_procedures, num_medications, number_diagnoses, diag_1, diag_2, diag_3, readmitted, admission_source_id, diabetesMed, insulin, medication_change, num_procedures, number_outpatient, number_emergency, number_inpatient) VALUES ('2278392', '8222157', 'Caucasian', 'Female', '[0-10)', '6', '25', '1', '41', '1', '1', '250.83', '?', '?', 'NO', '1', 'No', 'No', 'No', '0', '0', '0', '0');

INSERT INTO diabetes_readmissions (encounter_id, patient_nbr, race, gender, age, admission_type_id, discharge_disposition_id, time_in_hospital, num_lab_procedures, num_medications, number_diagnoses, diag_1, diag_2, diag_3, readmitted, admission_source_id, diabetesMed, insulin, medication_change, num_procedures, number_outpatient, number_emergency, number_inpatient) VALUES ('149190', '55629189', 'Caucasian',

## 5. Create Clean Table

In [48]:
with open("../sql/02_3_create_diabetes_readmissions_clean.sql", "r") as f:
    sql_script = f.read()

print("Create clean table SQL:", sql_script, sep="\n")
cursor.executescript(sql_script)

cursor.executescript(sql_script)

Create clean table SQL:
-- CLEAN TABLE - Contains cleaned data + engineered features

DROP TABLE IF EXISTS diabetes_readmissions_clean;

CREATE TABLE diabetes_readmissions_clean
(
    encounter_id NUMBER,
    patient_nbr NUMBER,
    race VARCHAR2 (100),
    gender VARCHAR2 (100),
    age VARCHAR2 (100),
    admission_type_id NUMBER,
    discharge_disposition_id NUMBER,
    time_in_hospital NUMBER,
    num_lab_procedures NUMBER,
    num_medications NUMBER,
    number_diagnoses NUMBER,
    diag_1 VARCHAR2(100),
    diag_2 VARCHAR2(100),
    diag_3 VARCHAR2(100),
    readmitted VARCHAR2(100),
    admission_source_id NUMBER,     
    medical_specialty VARCHAR2(100),
    diabetesMed VARCHAR2(100),     
    insulin VARCHAR2(100),       
    medication_change VARCHAR2(100),
    num_procedures NUMBER,
    number_outpatient NUMBER,
    number_emergency NUMBER,
    number_inpatient NUMBER,
    A1Cresult VARCHAR2(100),
    max_glu_serum VARCHAR2(100),
    readmit_30d NUMBER,
    los_group  VARCHA

## 6. Insert Data into Clean Table

In [50]:
with open("../sql/02_4_insert_diabetes_readmissions_clean.sql", "r") as f:
    sql_script = f.read()

print("Insert clean data SQL:", sql_script, sep="\n")

cursor.executescript(sql_script)

Insert clean data SQL:
DELETE FROM diabetes_readmissions_clean;

-- INSERT CLEANED AND FEATURE-ENGINEERED DATA
INSERT INTO diabetes_readmissions_clean (
    encounter_id,
    patient_nbr,
    race,
    gender,
    age,
    admission_type_id,
    discharge_disposition_id,
    time_in_hospital,
    num_lab_procedures,
    num_medications,
    number_diagnoses,
    diag_1,
    diag_2,
    diag_3,
    readmitted,
    admission_source_id,     
    medical_specialty,
    diabetesMed,     
    insulin,       
    medication_change,
    num_procedures,
    number_outpatient,
    number_emergency,
    number_inpatient,
    A1Cresult,
    max_glu_serum,
    readmit_30d,
    los_group,
    age_group,
    risk_score,
    risk_category
)
WITH d AS (
SELECT
    encounter_id,
    patient_nbr,
    CASE
        WHEN race = '?' THEN NULL
        ELSE race
    END AS race,
    gender,
    age,
    admission_type_id,
    discharge_disposition_id,
    time_in_hospital,
    num_lab_procedures,
    num_medic

In [51]:
df = pd.read_sql("SELECT * FROM diabetes_readmissions_clean", conn)
df.to_csv("../data/stage/readmissions_export.csv", index=False)
df

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,time_in_hospital,num_lab_procedures,num_medications,...,number_outpatient,number_emergency,number_inpatient,A1Cresult,max_glu_serum,readmit_30d,los_group,age_group,risk_score,risk_category
0,2278392,8222157,Caucasian,Female,0-10,6,25,1,41,1,...,0,0,0,None,None,0,Short,Young,0,Low
1,149190,55629189,Caucasian,Female,10-20,1,1,3,59,18,...,0,0,0,None,None,0,Short,Young,2,Medium
2,64410,86047875,African American,Female,20-30,1,1,2,11,13,...,2,0,1,None,None,0,Short,Young,2,Medium
3,500364,82442376,Caucasian,Male,30-40,1,1,2,44,16,...,0,0,0,None,None,0,Short,Middle,2,Medium
4,16680,42519267,Caucasian,Male,40-50,1,1,1,51,8,...,0,0,0,None,None,0,Short,Middle,2,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,2600796,1451637,Caucasian,Female,20-30,6,25,1,66,6,...,0,0,0,None,None,0,Short,Young,0,Low
195,2601576,279009,Caucasian,Female,60-70,6,25,1,31,16,...,0,0,0,None,None,0,Short,Senior,2,Medium
196,2602860,511965,Caucasian,Female,60-70,6,25,4,57,17,...,0,0,0,None,None,0,Medium,Senior,2,Medium
197,2629110,2283327,African American,Female,40-50,6,25,6,60,27,...,0,0,0,None,None,1,Medium,Middle,2,Medium


## 7. Analyze Data

In [36]:
with open("../sql/02_5_analytical_queries.sql", "r") as f:
    sql_script = f.read()

print("Read data SQL:", sql_script, sep="\n")

df = pd.read_sql(sql_script, conn)

df

Read data SQL:
-- Calculates the overall 30-day readmission rate (%)
-- This metric represents the average probability of a patient being readmitted
-- within 30 days after discharge across the entire dataset.
SELECT ROUND(AVG(readmit_30d)*100,1) as readmission_rate_pct
FROM diabetes_readmissions_clean;


















,readmission_rate_pct
0,8.0


In [37]:
with open("../sql/02_6_analytical_queries.sql", "r") as f:
    sql_script = f.read()

print("Read data SQL:", sql_script, sep="\n")

df = pd.read_sql(sql_script, conn)

df

Read data SQL:
-- Calculates readmission rate by length-of-stay group
-- Helps identify whether longer or shorter hospital stays
-- are associated with higher readmission risk.
SELECT los_group,ROUND(AVG(readmit_30d)*100,1) AS readmission_rate_pct
FROM diabetes_readmissions_clean
GROUP BY los_group
ORDER BY readmission_rate_pct DESC;


,los_group,readmission_rate_pct
0,Medium,11.4
1,Long,7.5
2,Short,5.0


In [38]:
with open("../sql/02_7_analytical_queries.sql", "r") as f:
    sql_script = f.read()

print("Read data SQL:", sql_script, sep="\n")

df = pd.read_sql(sql_script, conn)

df

Read data SQL:
-- Calculates readmission rate by age group
-- Used to assess how readmission risk changes across patient age segments.
SELECT age_group, ROUND(AVG(readmit_30d)*100,1) AS readmission_rate_pct
FROM diabetes_readmissions_clean
GROUP BY age_group;


,age_group,readmission_rate_pct
0,Middle,7.9
1,Senior,8.8
2,Young,0.0


In [39]:
with open("../sql/02_8_analytical_queries.sql", "r") as f:
    sql_script = f.read()

print("Read data SQL:", sql_script, sep="\n")

df = pd.read_sql(sql_script, conn)

df

Read data SQL:
-- Calculates readmission rate by discharge disposition
-- Identifies discharge pathways associated with higher readmission risk.
SELECT discharge_disposition_id, ROUND(AVG(readmit_30d)*100,1) AS readmission_rate_pct
FROM diabetes_readmissions_clean
GROUP BY discharge_disposition_id
ORDER BY readmission_rate_pct DESC;


,discharge_disposition_id,readmission_rate_pct
0,5,33.3
1,2,28.6
2,3,25.0
3,1,6.8
4,6,5.6
5,25,3.4
6,11,0.0
7,7,0.0


In [40]:
with open("../sql/02_9_analytical_queries.sql", "r") as f:
    sql_script = f.read()

print("Read data SQL:", sql_script, sep="\n")

df = pd.read_sql(sql_script, conn)

df

Read data SQL:
---- Calculates readmission rate by risk score
SELECT risk_score,
       ROUND(AVG(readmit_30d)*100,1) AS readmission_rate_pct,
       COUNT(*) AS patient_count
FROM diabetes_readmissions_clean
GROUP BY risk_score
ORDER BY risk_score;



,risk_score,readmission_rate_pct,patient_count
0,0,3.7,27
1,2,8.8,136
2,4,8.3,36


## 8. Save Changes

In [41]:
conn.commit()

conn.close()